In [64]:
import numpy as np

import ufl
from ufl import ds, dx, grad, inner

from dolfinx import fem, io, mesh, plot, log
from dolfinx.io import gmshio # to import gmsh functions
from dolfinx import geometry # to define the line plot
from dolfinx import default_scalar_type
import dolfinx.fem.petsc
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from mpi4py import MPI

from petsc4py.PETSc import ScalarType

import matplotlib.pyplot as plt

In [65]:
class LinearElasticity:
    def __init__(self, mesh_file, rank=0, gdim=3):
        """
        Initialize the LinearElasticity class with the mesh file.

        Parameters:
        mesh_file (str): Path to the mesh file.
        rank (int): Rank of the process (default is 0).
        gdim (int): Geometric dimension of the mesh (default is 3).
        """
        self.mesh_file = mesh_file
        self.rank = rank
        self.gdim = gdim
        self._read_mesh()
        self._setup_function_space()
        self._initialize_functions()

    def _read_mesh(self):
        """
        Read the mesh and facet tags from the specified mesh file.
        """
        try:
            self.domain, self.tags, self.ft = gmshio.read_from_msh(self.mesh_file, MPI.COMM_WORLD, rank=self.rank, gdim=self.gdim)
            self.tdim = self.domain.topology.dim
            self.fdim = self.tdim - 1
            self.domain.topology.create_connectivity(self.fdim, self.tdim)
        except Exception as e:
            raise ValueError(f"Error reading mesh file {self.mesh_file}: {e}")

    def _setup_function_space(self):
        """
        Set up the function space for the problem.
        """
        # self.V = fem.functionspace(self.domain, ("Lagrange", 1, (self.domain.geometry.dim,)))
        self.V = V = fem.functionspace(self.domain, ("Lagrange", 2, (self.domain.geometry.dim, )))

    def _initialize_functions(self):
        """
        Initialize trial and test functions.
        """
        self.u = fem.Function(self.V)
        self.v = ufl.TestFunction(self.V)

    def parameters(self, rho, g, lmbda, mu):
        """
        Set material and physical parameters.

        Parameters:
        rho (float): Density.
        g (float): Gravitational acceleration.
        lmbda (float): First Lamé parameter.
        mu (float): Second Lamé parameter.
        """
        self.rho = rho
        self.g = g
        self.lmbda = lmbda
        self.mu = mu

    def boundary_conditions(self, internal_pressure, outer_pressure):
        """
        Define and set the boundary conditions.
        """
        
        # Locate dofs for Dirichlet boundary conditions
        self.dofs_D_top     = fem.locate_dofs_topological(V=self.V, entity_dim=self.fdim, entities=self.ft.find(7))
        self.dofs_D_bottom  = fem.locate_dofs_topological(V=self.V, entity_dim=self.fdim, entities=self.ft.find(8))
        self.dofs_D_outer   = fem.locate_dofs_topological(V=self.V, entity_dim=self.fdim, entities=self.ft.find(9))
        self.dofs_D_inner   = fem.locate_dofs_topological(V=self.V, entity_dim=self.fdim, entities=self.ft.find(10))

        # Dirichlet boundary conditions
        self.bc_top     = fem.dirichletbc(np.array([0, 0, 0], dtype=default_scalar_type), dofs=self.dofs_D_top, V=self.V)
        self.bc_bottom  = fem.dirichletbc(np.array([0, 0, 0], dtype=default_scalar_type), dofs=self.dofs_D_bottom, V=self.V)
        self.bc_outer   = fem.dirichletbc(np.array([0, 0, 0], dtype=default_scalar_type), dofs=self.dofs_D_outer, V=self.V)
        self.bc_inner   = fem.dirichletbc(np.array([0, 0, 0], dtype=default_scalar_type), dofs=self.dofs_D_inner, V=self.V)

        self.bc = [self.bc_top, self.bc_bottom]

        # Neumann boundary conditions
        self.normal = ufl.FacetNormal(self.domain)
        self.pressure_inner = fem.Constant(self.domain, default_scalar_type(internal_pressure))
        self.pressure_outer = fem.Constant(self.domain, default_scalar_type(outer_pressure))
        
        self.traction_inner = self.pressure_inner * self.normal
        self.traction_outer = self.pressure_outer * self.normal

    def assemble_system(self):
        """
        Assemble the system for the linear elasticity problem.
        """
        def sigma(u, lmbda, mu):
            return lmbda * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)

        def epsilon(u):
            return ufl.sym(ufl.grad(u))

        # Body forces
        self.f = fem.Constant(self.domain, ScalarType((0.0, 0.0, - self.rho * self.g)))

        self.ds = ufl.Measure("ds", domain=self.domain, subdomain_data=self.ft)
        ds_in = self.ds(10)
        ds_out = self.ds(9)

        self.a = ufl.inner(sigma(self.u, self.lmbda, self.mu), epsilon(self.v)) * ufl.dx
        self.L = ufl.dot(self.f, self.v) * ufl.dx + ufl.dot(self.traction_outer, self.v) * ds_out + ufl.dot(self.traction_inner, self.v) * ds_in

        self.F = self.a - self.L

    def solve(self):
        """
        Solve the non linear elasticity problem.
        """       
        self.nonLinearProblem = NonlinearProblem(self.F, self.u, self.bc)

    def get_results(self):
        """
        Get the computed displacement, stress, and strain fields.

        Returns:
        displacement (Function): The displacement field.
        stress_field (ufl.Expr): The stress field.
        strain_field (ufl.Expr): The strain field.
        """
        return self.displacement, self.stress_field, self.strain_field

In [66]:
# Scaled variable
E = 1e9 # Pa
nu = 0.33
G = E / (2*(1+nu))
lame1 = nu * E /((1+nu)*(1-2*nu))
lame2 = G

internal_pressure = -1e7
outer_pressure = 0

print(f"Young's modulus = {E*1e-9:.2f} GPa")
print(f"Poisson's ratio = ", nu)
print(f"Shear modulus = {G*1e-9:.2f} GPa")
print(f"Lamé first parameter = {lame1*1e-9:.2f} GPa")
print(f"Lamé second parameter = {lame2*1e-9:.2f} GPa")

rho = 9560 # kg/m3
g = 9.81

problem = LinearElasticity(mesh_file='./cylindrical_shell.msh')

# Volume of the mesh = volume integral
dx_subdomain = ufl.Measure("dx", domain=problem.domain)
volume = fem.assemble_scalar(fem.form(1 * dx_subdomain))
print("---------")
print(f"Caclulated volume = ", volume)
print(f"Analytical volume = {3.14*(5**2 - 4.5**2)*4}")

problem.parameters(rho, g / volume, lame1, lame2)
problem.boundary_conditions(internal_pressure=internal_pressure, outer_pressure=outer_pressure)
problem.assemble_system()
problem.solve()

# Set Newton solver options
solver = NewtonSolver(MPI.COMM_WORLD, problem.nonLinearProblem)

solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"



Young's modulus = 1.00 GPa
Poisson's ratio =  0.33
Shear modulus = 0.38 GPa
Lamé first parameter = 0.73 GPa
Lamé second parameter = 0.38 GPa
Info    : Reading './cylindrical_shell.msh'...
Info    : 15 entities
Info    : 1386 nodes
Info    : 7056 elements
Info    : Done reading './cylindrical_shell.msh'
---------
Caclulated volume =  59.66881083605438
Analytical volume = 59.660000000000004


2024-07-30 09:44:49.622 (1030.486s) [main            ]         graphbuild.cpp:356   INFO| Build local part of mesh dual graph
2024-07-30 09:44:49.626 (1030.490s) [main            ]           ordering.cpp:204   INFO| GPS pseudo-diameter:(69) 2875-0
2024-07-30 09:44:49.627 (1030.491s) [main            ]           Topology.cpp:1330  INFO| Create topology (single cell type)
2024-07-30 09:44:49.627 (1030.491s) [main            ]           Topology.cpp:1044  INFO| Create topology (generalised)
2024-07-30 09:44:49.627 (1030.492s) [main            ]                MPI.cpp:164   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 1
2024-07-30 09:44:49.627 (1030.492s) [main            ]                MPI.cpp:235   INFO| Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
2024-07-30 09:44:49.628 (1030.493s) [main            ]          partition.cpp:233   INFO| Compute ghost indices
2024-07-30 09:44:49.628 (1030.493s) [main          

In [67]:
import pyvista

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(problem.u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(problem.u)] = problem.u.x.array.reshape(geometry.shape[0], len(problem.u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(problem.domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([problem.u[i]**2 for i in range(len(problem.u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    num_its, converged = solver.solve(problem.u)
    assert (converged)
    problem.u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}")
    function_grid["u"][:, :len(problem.u)] = problem.u.x.array.reshape(geometry.shape[0], len(problem.u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()


2024-07-30 09:44:52.758 (1033.623s) [main            ]      dofmapbuilder.cpp:166   INFO| Checking required entities per dimension
2024-07-30 09:44:52.759 (1033.623s) [main            ]      dofmapbuilder.cpp:264   INFO| Cell type:0, dofmap:5040x10
2024-07-30 09:44:52.760 (1033.624s) [main            ]      dofmapbuilder.cpp:320   INFO| Global index computation
2024-07-30 09:44:52.760 (1033.624s) [main            ]      dofmapbuilder.cpp:637   INFO| Got 2 index_maps
2024-07-30 09:44:52.760 (1033.624s) [main            ]      dofmapbuilder.cpp:644   INFO| Get global indices
2024-07-30 09:44:52.936 (1033.800s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:44:56.861 (1037.725s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:00.312 (1041.176s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 4.39089e-12 (tol = 1e-08) r 

Time step 1, Number of iterations 2


2024-07-30 09:45:00.796 (1041.660s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:04.355 (1045.219s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.


Time step 2, Number of iterations 2


2024-07-30 09:45:07.734 (1048.599s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 2.36664e-15 (tol = 1e-08) r (rel) = 0.913267(tol = 1e-08)
2024-07-30 09:45:07.734 (1048.599s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.
2024-07-30 09:45:08.071 (1048.936s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:11.612 (1052.477s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:14.754 (1055.619s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 2.37148e-15 (tol = 1e-08) r (rel) = 1.02394(tol = 1e-08)
2024-07-30 09:45:14.754 (1055.619s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


...

Time step 7, Number of iterations 2


2024-07-30 09:45:43.331 (1084.195s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:46.758 (1087.622s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:49.983 (1090.848s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 2.34991e-15 (tol = 1e-08) r (rel) = 0.953555(tol = 1e-08)
2024-07-30 09:45:49.983 (1090.848s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 8, Number of iterations 2


2024-07-30 09:45:50.318 (1091.182s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:53.927 (1094.792s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-30 09:45:57.213 (1098.078s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 2.48156e-15 (tol = 1e-08) r (rel) = 1.05716(tol = 1e-08)
2024-07-30 09:45:57.213 (1098.078s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 9, Number of iterations 2
